This trains the overall classifier

In [6]:
leg_corpus_name = 'NHlegtext09_20.corpus'
overall_data_name = 'NHLegOverallData09_20.csv'
win_state_list_name = 'NHWinList09_20.list'

In [25]:
import pickle
import numpy as np
from gensim.models import Doc2Vec as d2v
from gensim.models.doc2vec import TaggedDocument as td
import pandas as pd
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif

from pprint import pprint
from time import time
import logging

import warnings
from sklearn.metrics import classification_report
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore", ConvergenceWarning)

In [7]:
#load all of our documents
with open(leg_corpus_name,'rb') as f:
    legcorpus = pickle.load(f)
    
leg_data = pd.read_csv(overall_data_name)

with open(win_state_list_name,'rb') as f:
    win_state = pickle.load(f)

In [17]:
combined_docs = {}
total = 0
passed = 0
leg_target = {}

#for each document, check if it is approved, record that, then combine the list into a string
for doc in legcorpus:
    pass_flag = 0
    if doc[0] not in combined_docs:
        total += 1
        combined_docs[doc[0]] = " ".join(doc[2])
#         if total%1000 == 0:
#             print(total)

#for now, ignoring the amendments, but it seems like it could do better with the amendments
#print(list(combined_docs.keys())[0:10])

In [18]:
leg_targ = pd.DataFrame({"every stage passed":win_state}) 

bill_order = []
bill_target = []
bill_texts = []

leg_target = {}

for i, bill_id in enumerate(leg_data["bill_id"]):
    if str(bill_id) not in bill_order:
        bill_order.append(str(bill_id))
        bill_target.append(leg_targ["every stage passed"][i])
        leg_target[str(bill_id)] = leg_targ["every stage passed"][i]
        
        if bill_id in combined_docs:
            bill_texts.append(combined_docs[bill_id])
        else:
            bill_texts.append("")

In [20]:
features = leg_data
#we do need to assemble all of the data to train the model, and that is best done as close to the training as possible
X = pd.DataFrame(features)
X['leg_targ'] = leg_targ

X = X.drop_duplicates(subset=['title']) #should probably be bill_id
X = X.set_index('bill_id')
X = X.drop(columns=['time_t', 'event'])

bill_order = []
bill_target = []
bill_texts = []

count = 0
for bill_id in X.index:
    if bill_id in combined_docs:
        bill_texts.append(combined_docs[bill_id])
    else:
        bill_texts.append("")
        
X['texts'] = bill_texts
X = X.astype({'year': 'int64'}).sort_values('year', ascending=True)
X['month_introduced'] = X['month_introduced'].astype(str)

In [43]:
X_new = X[['sponsor_vec','bill_type', 'proposed_chamber',
           'primary_name','primary_party','number_sponsors',
           'committee_introduced','month_introduced', 
           #these features could be useful with more data
           #'primary_leadership','primary_ideology','public_sentiment','press',
          ]].copy()

#this dataset is slightly different than the original
amount_train = 7330
amount_test = 1000

X_dum = pd.get_dummies(X_new)

#depending on the format, you can use 
X_train_p = X_dum.iloc[0:amount_train]#.iloc
y_train = X['leg_targ'].iloc[0:amount_train]#pd.get_dummies()
X_test_p = X_dum.iloc[amount_train:amount_train+amount_test]#.iloc
y_test = X['leg_targ'].iloc[amount_train:amount_train+amount_test]#pd.get_dummies()

k_select = SelectKBest(f_classif, k=100)
X_train = k_select.fit_transform(X_train_p, y_train)
X_test = k_select.transform(X_test_p)

indices = k_select.get_support(indices=True)

print(sum(y_train)/len(y_train))
print(sum(y_test)/len(y_test))

0.3630286493860846
0.362


/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 11  17  24  30  33  37  41  58  72  77  93 103 105 117 119 120 135 145
 155 166 202 203 204 219 230 234 236 265 266 269 276 280 283 284 301 332
 354 385 388 407 409 414 417 418 424 461 464 471 493 507 527 567 570 590
 638 640 643 647 664 674 675 678 689 695 696 698 704 705 728 729 732 748
 758 773 812 817] are constant.
  UserWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [44]:
compressed_legtext = pd.DataFrame(X['texts'].copy())
#prepare the documents by using tfidf vectors:

#from https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a
stemmer = SnowballStemmer("english", ignore_stopwords=True)
#adjust this stemmer so that it does more than one word in the tokens

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])

stemmed_count_vect = StemmedCountVectorizer(stop_words='english', ngram_range=(1,4))
stem_counts = stemmed_count_vect.fit_transform(compressed_legtext['texts'][0:amount_train])
stem_counts_test = stemmed_count_vect.transform(compressed_legtext['texts'][amount_train:amount_train+amount_test])

tfidf = TfidfTransformer()
tfidf_vectors = tfidf.fit_transform(stem_counts)
tfidf_vectors_test = tfidf.transform(stem_counts_test)

In [23]:
#do the final pick after, since we only want to run the above cell once
k_best_tfidf = SelectKBest(f_classif, k=1000)
final_tfidf_features = k_best_tfidf.fit_transform(tfidf_vectors, y_train)
final_tfidf_features_test = k_best_tfidf.transform(tfidf_vectors_test)
word_indices = k_best_tfidf.get_support(indices=True)

In [24]:
#combine tfidf legtext and the other features:
legtext_train = final_tfidf_features.todense()

legtext_test = final_tfidf_features_test.todense()

X_train = np.concatenate([np.array(X_train),legtext_train], axis=1)
X_test = np.concatenate([np.array(X_test),legtext_test], axis=1)

In [45]:
means = []
n = 1
for i in range(n):
    clf = MLPClassifier(hidden_layer_sizes=(50,),
                        activation='relu',
                        random_state=i+100, max_iter=50,
                        solver='lbfgs', alpha=0.01,
                        tol=0.001, n_iter_no_change=4)
    #The MNB classifier can be useful too
    #clf = MultinomialNB(alpha=0)#alpha=0.001,DecisionTreeClassifier()#MultinomialNB()##19 ##
    
    clf.fit(X_train, y_train)
    predicted = clf.predict(X_test)
    predicted_prob = clf.predict_proba(X_test)
    new_mean = np.mean(predicted == y_test)
    means.append(new_mean)
    
    if n > 1:
        print(new_mean)
    
mean=np.mean(np.array(means))
print("overall mean:", mean)
print(classification_report(y_test, predicted, digits=4))

overall mean: 0.973
              precision    recall  f1-score   support

           0     0.9766    0.9812    0.9789       638
           1     0.9666    0.9586    0.9626       362

    accuracy                         0.9730      1000
   macro avg     0.9716    0.9699    0.9707      1000
weighted avg     0.9730    0.9730    0.9730      1000



In [46]:
#save the classifier
from joblib import dump, load
dump(clf, 'overall_clf.joblib')

['overall_clf.joblib']

In [48]:
#example of loading a classifier
clf = load('overall_clf.joblib') 
predicted = clf.predict(X_test)
new_mean = np.mean(predicted == y_test)
print(new_mean)

0.973
